In [1]:
# Imports
from luigi.contrib.spark import PySparkTask
from luigi.parameter import IntParameter, DateSecondParameter
from luigi import LocalTarget, Task, WrapperTask
import datetime
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
%run RulebasedClassifier.ipynb

class Evaluator(Task):

    # Date for Output-File prefix
    from datetime import date, timedelta
    date = DateSecondParameter(default=datetime.datetime.now())
    
    # Method to declare the Output-File
    def output(self):
        prefix = self.date.strftime("%Y-%m-%dT%H%M%S")
        return LocalTarget("data/%s_Evaluator_out.csv" % prefix, format=UTF8)
    
    # Method to define the required Task (Preprocessor)
    def requires(self):
        return RulebasedClassifier()


    # Classify the imported Data
    def run(self):
        df = pd.read_csv(self.input().path)
        print('TP: %s\tFP: %s\nFN: %s\tTN: %s'% self.calculateScore(df))
        print(classification_report(df['specified'].values, df['predicted'].values))
        
        # Write .csv-File
        with self.output().open("w") as out:
            df.to_csv(out, encoding="utf-8")
            
    def calculateScore(self, df):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for index, document in df.iterrows():
            if document.specified == 1 and document.predicted == 1:
                tp+=1
            elif document.specified == 0 and document.predicted == 1:
                fp+=1
            elif document.specified == 1 and document.predicted == 0:
                fn+=1
            else:
                tn+=1
        
        return tp, fp, fn, tn
            
evaluator = Evaluator()
evaluator.run()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


TP: 310	FP: 88
FN: 147	TN: 478
              precision    recall  f1-score   support

           0       0.76      0.84      0.80       566
           1       0.78      0.68      0.73       457

   micro avg       0.77      0.77      0.77      1023
   macro avg       0.77      0.76      0.76      1023
weighted avg       0.77      0.77      0.77      1023

